## Funciones de ML item-item

In [57]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

In [58]:
df_games = pd.read_csv('df_games_id.csv')
df_user = pd.read_csv('df_user_id.csv')
df_reviews = pd.read_csv('df_user_reviews.csv')

### Preprocesamiento de datos:
Vamos a procesar la columna 'genres' para que sea más fácil de trabajar y eliminar duplicados.

In [62]:
# Crear un DataFrame con la relación entre juegos y géneros
df_game_genres = pd.DataFrame({'id': df_games['id'], 'genres': df_games['genres'], 'app_name': df_games['app_name']})

# Eliminar duplicados
df_game_genres = df_game_genres.drop_duplicates(subset=['id', 'genres'])

### Matriz de géneros:
Construir una matriz que represente la relación entre los juegos y los géneros.

In [64]:
# Eliminar duplicados en df_game_genres
df_game_genres = df_game_genres.drop_duplicates(subset=['app_name', 'genres'])

# Crear la matriz de géneros
genres_matrix = df_game_genres.pivot(index='app_name', columns='genres', values='id').fillna(0)

### Similitud del coseno entre juegos:
Calcular la similitud del coseno entre juegos basada en la matriz de géneros.

In [50]:
cosine_sim = cosine_similarity(genres_matrix, genres_matrix)

### Función para obtener recomendaciones de juegos:

In [65]:
# Función para obtener recomendaciones de juegos similares
def get_recommendations_game(game_id, cosine_sim=cosine_sim):
    # Obtener la fila correspondiente al juego
    game_row = genres_matrix.loc[game_id].values.reshape(1, -1)
    
    # Calcular la similitud del coseno entre el juego y todos los demás juegos
    sim_scores = cosine_similarity(game_row, genres_matrix.values)
    
    # Obtener los juegos más similares (excluyendo el juego de entrada)
    similar_games = sim_scores.argsort()[0][::-1][1:]
    
    # Tomar los primeros 5 juegos recomendados
    recommendations = genres_matrix.index[similar_games][:5].tolist()
    
    return recommendations

### Prueba de uso:

In [68]:
# Ejemplo de uso con el id de un juego específico
game_id_ejemplo = 'Russian Roads'  # Reemplaza con un id real de juego
game_recommendations = get_recommendations_game(game_id_ejemplo)
print(f"Recomendaciones para el juego {game_id_ejemplo}: {game_recommendations}")

Recomendaciones para el juego Russian Roads: ['Russian Roads', 'RC-AirSim - RC Model Airplane Flight Simulator', 'Try Hard Parking', 'Car Mechanic Simulator 2015 - Total Modifications', 'SoundTracks: The Train Set Game']
